<a href="https://colab.research.google.com/github/thai94/d2l/blob/main/5.deep_learning_computation/5_1_layers_and_blocks_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import tensorflow as tf

In [28]:
class MLP(tf.keras.Model):

  def __init__(self):
    
    super().__init__()
    self.hidden = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)
    self.out = tf.keras.layers.Dense(units=10)
  
  def call(self, X):

    return self.out(self.hidden(X))

In [29]:
net = MLP()

X = tf.random.uniform((2, 20))
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.13604157, -0.40128654, -0.02873796, -0.1430833 ,  0.23163673,
         0.03409789,  0.36398298, -0.23955639, -0.36443013,  0.01839301],
       [-0.16784005, -0.44619405, -0.1423726 , -0.02055031,  0.3056544 ,
        -0.02499595,  0.3686195 , -0.08003469, -0.25312567,  0.08006692]],
      dtype=float32)>

In [30]:
class MySequential(tf.keras.Model):

  def __init__(self, *args):
    super().__init__()
    self.modules = []
    for block in args:
      self.modules.append(block)

  def call(self, X):
    for module in self.modules:
      X = module(X)
    return X

In [31]:
net = MySequential(
    tf.keras.layers.Dense(units=256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10))
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.09877494, -0.0742365 , -0.0236119 , -0.24790406,  0.14509954,
        -0.17161751, -0.00129081, -0.20132582, -0.04455721, -0.18625884],
       [ 0.03813782, -0.03675562,  0.09200394, -0.24980783,  0.05050098,
        -0.13002515,  0.08048572, -0.18704483, -0.27378413, -0.29266283]],
      dtype=float32)>

In [32]:
class FixedHiddenMLP(tf.keras.Model):
  def __init__(self):
    super().__init__()
    self.flatten = tf.keras.layers.Flatten()
    self.rand_weight = tf.constant(tf.random.uniform((20, 20)))
    self.dense = tf.keras.layers.Dense(20, activation=tf.nn.relu)

  def call(self, inputs):
    X = self.flatten(inputs)
    X = tf.nn.relu(tf.matmul(X, self.rand_weight) + 1)
    X = self.dense(X)
    while tf.reduce_sum(tf.math.abs(X)) > 1:
      X /= 2
    return tf.reduce_sum(X)

In [33]:
net = FixedHiddenMLP()
net(X)

<tf.Tensor: shape=(), dtype=float32, numpy=0.5036471>

In [34]:
class NestMLP(tf.keras.Model):
  
  def __init__(self):
    super().__init__()
    self.net = tf.keras.Sequential()
    self.net.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))
    self.net.add(tf.keras.layers.Dense(32, activation=tf.nn.relu))
    self.dense = tf.keras.layers.Dense(16, activation=tf.nn.relu)
  def call(self, inputs):
    return self.dense(self.net(inputs))

In [35]:
chimera = tf.keras.Sequential()
chimera.add(NestMLP())
chimera.add(tf.keras.layers.Dense(20))
chimera.add(FixedHiddenMLP())
chimera(X)

<tf.Tensor: shape=(), dtype=float32, numpy=0.6944176>